In [16]:
# 096: noise; 104-1 channel 2 some segments, channel 1 noise all
# 60, 65, 67 (day 5), 83, 91 (ngay 5 bo file 1), 94, 97 (bị ít), 106(1 bi it, 5 nhieu), 107: patterned irregularities 

# 65, 106-5: not included
# 73.1 73.5 noise (hoac nhip tim thay doi dot ngot, ko regular)

In [17]:
import pandas as pd
import random
import numpy as np
import os

In [18]:
noise_dict = {
    '096-1.csv': ['# ECG Channel 1'],
    '096-5.csv': ['# ECG Channel 1'],
    '104-5.csv': ['# ECG Channel 1', 'ECG Channel 2'],
    '060-5.csv': ['# ECG Channel 1'],
    '065-5.csv': ['# ECG Channel 1'],
    '067-5.csv': ['# ECG Channel 1'],
    '083-1.csv': ['# ECG Channel 1'],
    '091-5.2.csv': ['# ECG Channel 1'],
    '094-1.csv': ['# ECG Channel 1'],
    '094-5.csv': ['# ECG Channel 1'],
    '097-5.csv': ['# ECG Channel 1'],
    '106-1.csv': ['# ECG Channel 1'],
    '106-5.2.csv': ['# ECG Channel 1'],
    '107-5.csv': ['# ECG Channel 1'],
    '073-1.csv': ['# ECG Channel 1'],
    '073-5.csv': ['# ECG Channel 1']
}
FILE_PATH = '/media/tuan/NGUI GA/ECG_cleaned'
DATA_POINTS = 7680
SAMPLE_PER_FILE = 15
FILE_COUNT = 0
for entry in noise_dict:
    FILE_COUNT += len(noise_dict[entry])

In [19]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=123, shuffle=True)
file_count_range = list(range(1, FILE_COUNT * SAMPLE_PER_FILE + 1, 1))
kf.get_n_splits(file_count_range)

5

In [20]:
folds = list(kf.split(file_count_range))

In [21]:
folds = list(map(lambda f: f[1], folds))

In [22]:
folds

[array([  4,   5,  19,  20,  24,  29,  31,  33,  37,  42,  52,  53,  71,
         79,  80,  85,  89, 100, 107, 127, 128, 130, 136, 141, 148, 150,
        159, 167, 172, 173, 177, 181, 182, 190, 191, 192, 196, 198, 200,
        202, 206, 218, 219, 223, 231, 232, 235, 236, 242, 244, 248]),
 array([  8,  10,  11,  21,  23,  26,  35,  41,  50,  54,  61,  62,  72,
         77,  81,  82,  90,  91,  95, 112, 119, 120, 122, 125, 142, 147,
        151, 152, 156, 157, 160, 161, 166, 170, 171, 175, 183, 188, 197,
        199, 204, 205, 209, 212, 213, 221, 222, 229, 239, 243, 247]),
 array([  0,   7,  13,  16,  27,  28,  30,  36,  38,  40,  44,  45,  46,
         59,  60,  63,  65,  74,  88,  93, 104, 108, 110, 114, 116, 121,
        132, 133, 137, 140, 143, 144, 145, 163, 165, 169, 178, 184, 185,
        186, 193, 201, 207, 210, 211, 215, 216, 217, 227, 238, 241]),
 array([  1,   3,   6,   9,  12,  14,  15,  18,  22,  25,  34,  43,  48,
         51,  56,  58,  64,  67,  69,  70,  75,  76,  86,  8

In [23]:
def get_fold(file_index):
    for i, f in enumerate(folds):
        if file_index in f:
            return i + 1
    print(file_index)
    return -1

In [24]:
def list_files(in_path):
    csv_files = []
    for (dirpath, dirnames, filenames) in os.walk(in_path):
        for file in filenames:
            filename, ext = os.path.splitext(file)
            ext = str.lower(ext)
            if ext == '.csv':
                csv_files.append(os.path.join(dirpath, file))
    csv_files.sort()
    return csv_files

In [25]:
def generate_random_n_number(n, max_n, min_difference=7680):
    return sorted(random.sample(range(0, max_n, min_difference), n))

In [26]:
generate_random_n_number(10, 300000, 7680)

[7680, 46080, 76800, 122880, 161280, 176640, 230400, 253440, 261120, 268800]

In [27]:
output_folder = './splitted/'
if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

In [28]:
output_folder = './splitted/output'
if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

In [29]:
for i in range(1,6):
    fold_path = f'./splitted/output/{i}'
    if not os.path.isdir(fold_path):
        os.mkdir(fold_path)

In [30]:
ground_truth = pd.DataFrame(columns=['file', 'begin', 'end', 'original', 'fold'])
file_index = 0
file_output_folder = f'./splitted/output'
if not os.path.isdir(file_output_folder):
    os.mkdir(file_output_folder)
for entry in noise_dict:
    full_file_path = f"{FILE_PATH}/{entry}"
    df = pd.read_csv(full_file_path)
    channels = noise_dict[entry]
    for channel in channels:
        print(entry, channels)
        channel_data = df[channel]
        max_channel_len = len(channel_data.values)
        random_timepoints = generate_random_n_number(SAMPLE_PER_FILE, max_channel_len - 10000, 7680)
        
        for time_index, timepoint in enumerate(random_timepoints):
            partial_df = channel_data[timepoint: timepoint + 7680]
            partial_filename = f'{file_index}.csv'
            
            file_fold = get_fold(file_index)
            file_output_folder = f'./splitted/output/{file_fold}'
            
            partial_df.to_csv(f'{file_output_folder}/{partial_filename}', header=False, index=False)
            ground_truth = ground_truth.append({
                'file': partial_filename,
                'begin': timepoint,
                'end': timepoint + 7680,
                'original': entry,
                'channel': channel,
                'fold': file_fold
            }, ignore_index=True)
            file_index += 1

ground_truth.to_csv(f'{output_folder}/gt.csv', index = False)

096-1.csv ['# ECG Channel 1']
096-5.csv ['# ECG Channel 1']
104-5.csv ['# ECG Channel 1', 'ECG Channel 2']
104-5.csv ['# ECG Channel 1', 'ECG Channel 2']
060-5.csv ['# ECG Channel 1']
065-5.csv ['# ECG Channel 1']
067-5.csv ['# ECG Channel 1']
083-1.csv ['# ECG Channel 1']
091-5.2.csv ['# ECG Channel 1']
094-1.csv ['# ECG Channel 1']
094-5.csv ['# ECG Channel 1']
097-5.csv ['# ECG Channel 1']
106-1.csv ['# ECG Channel 1']
106-5.2.csv ['# ECG Channel 1']
107-5.csv ['# ECG Channel 1']
073-1.csv ['# ECG Channel 1']
073-5.csv ['# ECG Channel 1']
